In [1]:
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
from jaxtyping import Float, Int

In [18]:
@dataclass
class GPTConfig:
	# default test values -- too small for a real language model, but big enough for testing
	d_vocab: int = 10_000
	d_model: int = 128
	d_mlp: int = 512
	n_heads: int = 4
	d_head: int = 32
	n_layers: int = 6
	act_fn: type[nn.Module] = nn.ReLU

	@property
	def n_params(self) -> int:
		"an estimate of the number of parameters"
		return (
			self.d_vocab * self.d_model # embeddings (and tied unembeddings)
			+ (
				self.d_model * self.d_mlp * 2 # mlp weights
				+ self.d_model + self.d_mlp # mlp bias
				+ self.n_heads * ( # number of heads
					4 * self.d_model * self.d_head # 4 because Q, K, O, V
				)
			) * self.n_layers, # for each layer
		)
	
print(GPTConfig().n_params)

# note: the residual stream is `n_context` by `d_model`

# this is the row-wise (last dimension) softmax of x
# F.softmax(x, dim=-1)

class AttentionHead(nn.Module):

	def __init__(self, cfg: GPTConfig):
		super().__init__()
		self.relu = nn.ReLU()
		self.d_vocab = cfg.d_vocab
		self.d_model = cfg.d_model
		self.d_head = cfg.d_head
		self.wq = nn.Linear(self.d_model, self.d_head)
		self.wk = nn.Linear(self.d_model, self.d_head)
		self.wv = nn.Linear(self.d_model, self.d_head)
		self.wo = nn.Linear(self.d_head, self.d_model)

	
	
	def forward(self, x: Int[torch.Tensor, "n_context d_model"]) -> Float[torch.Tensor, "n_context d_model"]:
		def masking_matrix(n_context):
			mask = torch.zeros((n_context, n_context))  # Start with all 0s
			mask[torch.triu(torch.ones((n_context, n_context)), diagonal=1) == 1] = -float('inf')  # Set above diagonal to -inf
			return mask
		
		M = masking_matrix(x.shape[0])
		# softmax_argument = x*self.wq*torch.transpose(self.wk)*torch.transpose(x) + M
		wk_out = torch.transpose(self.wk(x), 0, 1)
		print("WK shape ", wk_out.shape)
		wq_out = self.wq(x)
		print("WQ shape ", wq_out.shape)
		softmax_out = F.softmax((wq_out@wk_out + M), dim=-1)
		print("Softmax shape ", softmax_out.shape)
		wv_out = self.wv(x)
		print("WV shape ", wv_out.shape)
		wo_out = self.wo(wv_out)
		# wo_out = self.wo(wv_out)
		result = softmax_out@wo_out
		print("Final A Shape ", result.shape)
		return result
		
		
class MultiHeadedAttention(nn.Module):

    def __init__(self, cfg: GPTConfig):
        super().__init__()
        self.n_heads = cfg.n_heads
        self.d_model = cfg.d_model
        self.d_head = cfg.d_head
        
        # Multi-head attention
        self.attention_heads = nn.ModuleList([AttentionHead(cfg) for _ in range(self.n_heads)])

        # Linear projection to project summed outputs back to d_model
        self.wo = nn.Linear(self.d_model, self.d_model)  # Fix the output size

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        head_outputs = [head(x) for head in self.attention_heads]
        print("Head output shape: ", head_outputs[0].shape)

        summed_heads = torch.sum(torch.stack(head_outputs), dim=0)  # Sum over heads -> (n_context, d_head)

        summed_heads += x  # Element-wise addition (ensures same shape)

        # Project back to d_model
        output = self.wo(summed_heads)  # (n_context, d_model)

        return output





class MLP(nn.Module):

	def __init__(self, cfg: GPTConfig):
		super().__init__()
		raise NotImplementedError()

	def forward(x: Int[torch.Tensor, "n_context d_model"]) -> Float[torch.Tensor, "n_context d_model"]:
		raise NotImplementedError()



class Transformer(nn.Module):

	def __init__(self, cfg: GPTConfig):
		super().__init__()
		raise NotImplementedError()
		# uses `MultiHeadedAttention` and `MLP`
		# uses nn.Embedding for the embedding, transpose of it for the unembedding

	def forward(x: Int[torch.Tensor, "n_context"]) -> Float[torch.Tensor, "n_context d_vocab"]:
		raise NotImplementedError()

(2463488,)


In [19]:
gpt_config = GPTConfig()
attn_head = AttentionHead(gpt_config)
x = torch.randn(256, gpt_config.d_model)
# print(x)
# print(x.shape)
attn_head.forward(x)
multi_head = MultiHeadedAttention(gpt_config)
multi_head.forward(x).shape

WK shape  torch.Size([32, 256])
WQ shape  torch.Size([256, 32])
Softmax shape  torch.Size([256, 256])
WV shape  torch.Size([256, 32])
Final A Shape  torch.Size([256, 128])
WK shape  torch.Size([32, 256])
WQ shape  torch.Size([256, 32])
Softmax shape  torch.Size([256, 256])
WV shape  torch.Size([256, 32])
Final A Shape  torch.Size([256, 128])
WK shape  torch.Size([32, 256])
WQ shape  torch.Size([256, 32])
Softmax shape  torch.Size([256, 256])
WV shape  torch.Size([256, 32])
Final A Shape  torch.Size([256, 128])
WK shape  torch.Size([32, 256])
WQ shape  torch.Size([256, 32])
Softmax shape  torch.Size([256, 256])
WV shape  torch.Size([256, 32])
Final A Shape  torch.Size([256, 128])
WK shape  torch.Size([32, 256])
WQ shape  torch.Size([256, 32])
Softmax shape  torch.Size([256, 256])
WV shape  torch.Size([256, 32])
Final A Shape  torch.Size([256, 128])
Head output shape:  torch.Size([256, 128])
torch.Size([256, 128])
